In [ ]:
!git clone https://github.com/Unknown1502/NeurIPS.git /kaggle/working/NeurIPS
!ls -la /kaggle/working/NeurIPS
!ls /kaggle/working/NeurIPS/data | head -10


Cloning into 'NeurIPS'...
remote: Enumerating objects: 560, done.
remote: Counting objects: 100% (560/560), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 560 (delta 484), reused 560 (delta 484), pack-reused 0 (from 0)
Receiving objects: 100% (560/560), 2.59 MiB | 4.23 MiB/s, done.
Resolving deltas: 100% (484/484), done.


In [6]:
import json
import sys
from pathlib import Path

# ============================================================================
# PATH CONFIGURATION
# ============================================================================

# Repository path
# LOCAL:  repo_path = r'c:\Users\prajw\OneDrive\Desktop\google golf'
# KAGGLE: repo_path = '/kaggle/working/NeurIPS'

repo_path = '/kaggle/working/NeurIPS'  # KAGGLE PATH
sys.path.insert(0, repo_path)

# Data directory path (dataset is in the GitHub repo)
# LOCAL:  DATA_DIR = "./data"
# KAGGLE: DATA_DIR = "/kaggle/working/NeurIPS/data"

DATA_DIR = "/kaggle/working/NeurIPS/data"  # KAGGLE PATH

# Solutions output directory
# LOCAL:  SOLUTIONS_DIR = "./solutions"
# KAGGLE: SOLUTIONS_DIR = "/kaggle/working/solutions"

SOLUTIONS_DIR = "/kaggle/working/solutions"  # KAGGLE PATH

# ============================================================================
# IMPORTS
# ============================================================================

from arc_solver import ARCTaskSolver
from utils import grid_operations as go
from utils import pattern_detection as pd
from collections import Counter
from typing import List, Tuple, Dict

# Create solutions directory
Path(SOLUTIONS_DIR).mkdir(exist_ok=True, parents=True)

# ============================================================================
# VERIFY SETUP
# ============================================================================

print("="*70)
print("SETUP COMPLETE")
print("="*70)
print(f"Repository path: {repo_path}")
print(f"Data directory: {DATA_DIR}")
print(f"Solutions directory: {Path(SOLUTIONS_DIR).absolute()}")
print("="*70)


SETUP COMPLETE
Repository path: /kaggle/working/NeurIPS
Data directory: /kaggle/working/NeurIPS/data
Solutions directory: /kaggle/working/solutions


In [7]:
# Verify dataset exists and check structure
print("DATASET VERIFICATION")
print("="*60)

data_path = Path(DATA_DIR)
if data_path.exists():
    print(f"Data directory found: {data_path}")
    
    # Count task files
    task_files = list(data_path.glob("task*.json"))
    print(f"Total task files: {len(task_files)}")
    
    if task_files:
        # Show first few task files
        print(f"\nFirst 5 task files:")
        for i, task_file in enumerate(task_files[:5], 1):
            print(f"  {i}. {task_file.name}")
        
        # Check a sample task structure
        with open(task_files[0], 'r') as f:
            sample = json.load(f)
        
        print(f"\nSample task structure (task001.json):")
        print(f"  Keys: {list(sample.keys())}")
        if 'train' in sample:
            print(f"  Train examples: {len(sample['train'])}")
        if 'test' in sample:
            print(f"  Test examples: {len(sample['test'])}")
        if 'arc-gen' in sample:
            print(f"  Arc-gen examples: {len(sample.get('arc-gen', []))}")
        
        print("\nDataset verification successful!")
    else:
        print("ERROR: No task files found in data directory")
else:
    print(f"ERROR: Data directory not found: {data_path}")
    print("Make sure you have cloned the repository in the previous cell")

print("="*60)


DATASET VERIFICATION
Data directory found: /kaggle/working/NeurIPS/data
Total task files: 400

First 5 task files:
  1. task240.json
  2. task247.json
  3. task164.json
  4. task264.json
  5. task271.json

Sample task structure (task001.json):
  Keys: ['train', 'test', 'arc-gen']
  Train examples: 3
  Test examples: 1
  Arc-gen examples: 0

Dataset verification successful!


In [8]:
# Initialize the solver framework
solver = ARCTaskSolver(data_dir=DATA_DIR, solutions_dir=SOLUTIONS_DIR)

# Verify initialization
task_files = list(Path(DATA_DIR).glob("task*.json"))
print(f"Solver initialized with {len(task_files)} tasks")

if task_files:
    # Check first task
    with open(task_files[0], 'r') as f:
        sample = json.load(f)
    
    print(f"\nTask structure verified:")
    print(f"  Train examples: {len(sample.get('train', []))}")
    print(f"  Test examples: {len(sample.get('test', []))}")
    print(f"  Arc-gen examples: {len(sample.get('arc-gen', []))}")
    
    print("\nSolver initialized successfully!")
else:
    print("\nERROR: No task files found")
    print("Verify DATA_DIR path is correct")


Solver initialized with 400 tasks

Task structure verified:
  Train examples: 3
  Test examples: 1
  Arc-gen examples: 0

Solver initialized successfully!


In [9]:
# Select task to analyze
TASK_ID = 1

# Load task data
task_data = solver.load_task(TASK_ID)

# Print detailed analysis
solver.print_task_analysis(TASK_ID)

TASK 001 ANALYSIS
Train examples:   5
Test examples:    1
Arc-gen examples: 262
TOTAL examples:   268

TRAIN Examples:
  Example 1:
    Input:  3x3 grid
    Output: 9x9 grid
    Input grid:  [[0, 7, 7], [7, 7, 7], [0, 7, 7]]
    Output grid: [[0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7]]
  Example 2:
    Input:  3x3 grid
    Output: 9x9 grid
    Input grid:  [[4, 0, 4], [0, 0, 0], [0, 4, 0]]
    Output grid: [[4, 0, 4, 0, 0, 0, 4, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 0, 0, 0, 0, 4, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 0, 4, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 4, 0, 0, 0, 0]]

TEST Examples:
  Example 1:
    Input:  3x3 grid
    Output: 9x9 grid
    Input grid:  [[7, 0, 7], [7, 0, 7], 

In [10]:
def visualize_example(input_grid, output_grid, example_num=1):
    """Display an input-output pair."""
    print(f"\n{'='*60}")
    print(f"Example {example_num}")
    print(f"{'='*60}")
    
    print(f"\nInput ({len(input_grid)}x{len(input_grid[0]) if input_grid else 0}):")
    for row in input_grid:
        print("  ", row)
    
    print(f"\nOutput ({len(output_grid)}x{len(output_grid[0]) if output_grid else 0}):")
    for row in output_grid:
        print("  ", row)
    
    print()

# Visualize first 2 training examples
print("TRAINING EXAMPLES:")
for i, pair in enumerate(task_data['train'][:2], 1):
    visualize_example(pair['input'], pair['output'], i)

TRAINING EXAMPLES:

Example 1

Input (3x3):
   [0, 7, 7]
   [7, 7, 7]
   [0, 7, 7]

Output (9x9):
   [0, 0, 0, 0, 7, 7, 0, 7, 7]
   [0, 0, 0, 7, 7, 7, 7, 7, 7]
   [0, 0, 0, 0, 7, 7, 0, 7, 7]
   [0, 7, 7, 0, 7, 7, 0, 7, 7]
   [7, 7, 7, 7, 7, 7, 7, 7, 7]
   [0, 7, 7, 0, 7, 7, 0, 7, 7]
   [0, 0, 0, 0, 7, 7, 0, 7, 7]
   [0, 0, 0, 7, 7, 7, 7, 7, 7]
   [0, 0, 0, 0, 7, 7, 0, 7, 7]


Example 2

Input (3x3):
   [4, 0, 4]
   [0, 0, 0]
   [0, 4, 0]

Output (9x9):
   [4, 0, 4, 0, 0, 0, 4, 0, 4]
   [0, 0, 0, 0, 0, 0, 0, 0, 0]
   [0, 4, 0, 0, 0, 0, 0, 4, 0]
   [0, 0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 4, 0, 4, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 4, 0, 0, 0, 0]



In [11]:
# Gather input and output grids
input_grids = [pair['input'] for pair in task_data['train']]
output_grids = [pair['output'] for pair in task_data['train']]

# Detect transformation type
print("TRANSFORMATION CHARACTERISTICS:")
trans_type = pd.detect_transformation_type(input_grids, output_grids)
for key, value in trans_type.items():
    if value:
        print(f"  [X] {key.replace('_', ' ').title()}")

print("\n" + "="*60)
print("COMPLEXITY ANALYSIS:")
complexity = pd.analyze_task_complexity(input_grids, output_grids)
for key, value in complexity.items():
    print(f"  {key}: {value}")

print("\n" + "="*60)
print("SUGGESTED APPROACHES:")
suggestions = pd.suggest_approach(input_grids, output_grids)
for i, suggestion in enumerate(suggestions, 1):
    print(f"  {i}. {suggestion}")


TRANSFORMATION CHARACTERISTICS:
  [X] Changes Dimensions
  [X] Increases Size
  [X] Preserves Colors
  [X] Constant Output Size

COMPLEXITY ANALYSIS:
  max_input_size: 9
  max_output_size: 81
  unique_colors_input: 5
  unique_colors_output: 5
  dimension_changes: True
  variable_output_size: False

SUGGESTED APPROACHES:
  1. Scaling or expansion operation
  2. Output size is constant - may be counting or aggregation


In [12]:
# Example solution for Task 1 (tiling pattern)
# Pattern observed: Input is 3x3, output tiles it into 3x3 arrangement (9x9 total)

def solve_v1(grid):
    """
    Clear implementation - tile the input 3x3.
    This is the initial version focused on correctness.
    """
    # Tile horizontally 3 times
    tiled_horizontal = [row * 3 for row in grid]
    
    # Tile vertically 3 times
    result = tiled_horizontal * 3
    
    return result

# Test on first training example
test_input = task_data['train'][0]['input']
test_output = task_data['train'][0]['output']
result = solve_v1(test_input)

print("Testing initial solution on first training example:")
print(f"Input dimensions: {len(test_input)}x{len(test_input[0])}")
print(f"Expected output dimensions: {len(test_output)}x{len(test_output[0])}")
print(f"Actual output dimensions: {len(result)}x{len(result[0])}")
print(f"Match: {result == test_output}")

Testing initial solution on first training example:
Input dimensions: 3x3
Expected output dimensions: 9x9
Actual output dimensions: 9x9
Match: False


In [13]:
# Test solution comprehensively
is_correct, message, stats = solver.test_solution(solve_v1, task_data, verbose=False)

print("="*60)
print("COMPREHENSIVE TEST RESULTS")
print("="*60)
print(f"\nTrain examples: {stats['train']['passed']}/{stats['train']['total']}")
print(f"Test examples: {stats['test']['passed']}/{stats['test']['total']}")
print(f"Arc-gen examples: {stats['arc-gen']['passed']}/{stats['arc-gen']['total']}")

total_passed = sum(s['passed'] for s in stats.values())
total_tests = sum(s['total'] for s in stats.values())

print(f"\nTOTAL: {total_passed}/{total_tests} passed")
print(f"Success rate: {100 * total_passed / total_tests:.2f}%")

if is_correct:
    print("\nSUCCESS: SOLUTION IS CORRECT")
else:
    print(f"\nFAILURE: SOLUTION FAILED")
    print(f"Details: {message}")

COMPREHENSIVE TEST RESULTS

Train examples: 0/5
Test examples: 0/1
Arc-gen examples: 0/262

TOTAL: 0/268 passed
Success rate: 0.00%

FAILURE: SOLUTION FAILED
Details: FAILURE: 0/268 passed (268 failed)
  Failure 1: train[0] - Output mismatch
  Failure 2: train[1] - Output mismatch
  Failure 3: train[2] - Output mismatch
  ... and 265 more failures


In [14]:
# Optimized version - minimize byte count
solve = lambda g: [r*3 for r in g]*3

# Compare byte counts
v1_code = "def solve_v1(grid):\n    tiled_horizontal = [row * 3 for row in grid]\n    result = tiled_horizontal * 3\n    return result"
v2_code = "solve = lambda g: [r*3 for r in g]*3"

v1_bytes = len(v1_code.encode('utf-8'))
v2_bytes = len(v2_code.encode('utf-8'))

print("CODE GOLF OPTIMIZATION RESULTS:")
print("="*60)
print(f"\nInitial version (clear):")
print(f"  Byte count: {v1_bytes}")
print(f"  Score (if correct): {max(1, 2500 - v1_bytes)}")

print(f"\nOptimized version (golf):")
print(f"  Byte count: {v2_bytes}")
print(f"  Score (if correct): {max(1, 2500 - v2_bytes)}")

print(f"\nImprovement:")
print(f"  Bytes saved: {v1_bytes - v2_bytes}")
print(f"  Points gained: {max(1, 2500 - v2_bytes) - max(1, 2500 - v1_bytes)}")

# Verify optimized version still works
is_correct_opt, _, _ = solver.test_solution(solve, task_data, verbose=False)
print(f"\nOptimized version correctness: {'PASS' if is_correct_opt else 'FAIL'}")

CODE GOLF OPTIMIZATION RESULTS:

Initial version (clear):
  Byte count: 120
  Score (if correct): 2380

Optimized version (golf):
  Byte count: 36
  Score (if correct): 2464

Improvement:
  Bytes saved: 84
  Points gained: 84

Optimized version correctness: FAIL


In [15]:
# Final solution code (as it will be saved)
final_solution = """solve=lambda g:[r*3 for r in g]*3"""

# Process solution and save if correct
result = solver.process_solution(TASK_ID, final_solution, verbose=True)

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
print(f"Task ID: {result['task_id']}")
print(f"Correctness: {'PASS' if result['success'] else 'FAIL'}")
print(f"Byte count: {result['byte_count']}")
print(f"Score: {result['score']:.3f} points")

if result['success']:
    print(f"\nSolution saved to: solutions/task{TASK_ID:03d}.py")
else:
    print(f"\nSolution not saved (failed validation)")

PROCESSING SOLUTION FOR TASK 001
  [train] Example 1/5: FAIL
  [train] Example 2/5: FAIL
  [train] Example 3/5: FAIL
  [train] Example 4/5: FAIL
  [train] Example 5/5: FAIL
  [test] Example 1/1: FAIL

Test Results:
  Train:   0/5 passed
  Test:    0/1 passed
  Arc-gen: 0/262 passed
  TOTAL:   0/268 passed

Metrics:
  Correctness: FAIL
  Byte count:  33 bytes
  Score:       0.001 points

FAILURE: 0/268 passed (268 failed)
  Failure 1: train[0] - Output mismatch
  Failure 2: train[1] - Output mismatch
  Failure 3: train[2] - Output mismatch
  ... and 265 more failures

FINAL RESULTS
Task ID: 1
Correctness: FAIL
Byte count: 33
Score: 0.001 points

Solution not saved (failed validation)


In [16]:
# Batch analyze multiple tasks
START_TASK = 1
END_TASK = 10

print(f"Analyzing tasks {START_TASK} to {END_TASK}...")
print("="*60)

for task_id in range(START_TASK, END_TASK + 1):
    try:
        analysis = solver.analyze_task(task_id)
        print(f"Task {task_id:03d}: "
              f"train={analysis['train_count']}, "
              f"test={analysis['test_count']}, "
              f"arc-gen={analysis['arc_gen_count']}, "
              f"total={analysis['total_pairs']}")
    except Exception as e:
        print(f"Task {task_id:03d}: ERROR - {e}")

print("="*60)

Analyzing tasks 1 to 10...
Task 001: train=5, test=1, arc-gen=262, total=268
Task 002: train=5, test=1, arc-gen=262, total=268
Task 003: train=3, test=1, arc-gen=261, total=265
Task 004: train=2, test=1, arc-gen=262, total=265
Task 005: train=3, test=1, arc-gen=262, total=266
Task 006: train=3, test=1, arc-gen=262, total=266
Task 007: train=3, test=1, arc-gen=262, total=266
Task 008: train=3, test=1, arc-gen=262, total=266
Task 009: train=3, test=1, arc-gen=261, total=265
Task 010: train=2, test=1, arc-gen=262, total=265


In [17]:
# Display progress report
solver.report_progress()

PROGRESS REPORT
Tasks solved:     0/400 (0.0%)
Total score:      0.00 points



In [ ]:
# CORRECTED SOLUTION FOR TASK 1
# Pattern: Create a 3x3 tiling where each tile is either the input grid or zeros

def solve_corrected(g):
    """
    Each cell value in the input determines what goes in that tile position:
    - If cell is 0: place a 3x3 block of zeros
    - If cell is non-zero: place the input grid itself
    """
    n = len(g)
    result = []
    for i in range(n):
        for sub_row in range(n):
            row = []
            for j in range(n):
                if g[i][j] == 0:
                    row.extend([0] * n)
                else:
                    row.extend(g[sub_row])
            result.append(row)
    return result

# Test the corrected solution
TASK_ID = 1
task_data = solver.load_task(TASK_ID)

# First, manually verify with Example 1
print("Manual verification with Example 1:")
test_input = task_data['train'][0]['input']
test_output = task_data['train'][0]['output']
my_output = solve_corrected(test_input)

print(f"Input: {test_input}")
print(f"\nMy output matches expected: {my_output == test_output}")

if my_output != test_output:
    print("\nExpected first 3 rows:")
    for row in test_output[:3]:
        print(f"  {row}")
    print("\nGot first 3 rows:")
    for row in my_output[:3]:
        print(f"  {row}")

print("\n" + "="*60)

# Now test comprehensively
is_correct, message, stats = solver.test_solution(solve_corrected, task_data, verbose=False)

print("CORRECTED SOLUTION TEST")
print("="*60)
print(f"Train examples: {stats['train']['passed']}/{stats['train']['total']}")
print(f"Test examples: {stats['test']['passed']}/{stats['test']['total']}")
print(f"Arc-gen examples: {stats['arc-gen']['passed']}/{stats['arc-gen']['total']}")

total_passed = sum(s['passed'] for s in stats.values())
total_tests = sum(s['total'] for s in stats.values())

print(f"\nTOTAL: {total_passed}/{total_tests} passed")
print(f"Success rate: {100 * total_passed / total_tests:.2f}%")

if is_correct:
    print("\nSUCCESS: SOLUTION IS CORRECT")
    
    # Now optimize for code golf
    print("\n" + "="*60)
    print("CODE GOLF OPTIMIZATION")
    print("="*60)
    
    # Optimized version - more compact
    solve = lambda g: [[x for j in range(len(g)) for x in (g[s] if g[i][j] else [0]*len(g))] for i in range(len(g)) for s in range(len(g))]
    
    # Alternative even shorter
    solve2 = lambda g:sum([[[0]*len(g)if g[i][j]==0 else g[s]for j in range(len(g))]for i in range(len(g))for s in range(len(g))],[])
    
    # Test both optimized versions
    is_opt1_correct, _, _ = solver.test_solution(solve, task_data, verbose=False)
    is_opt2_correct, _, _ = solver.test_solution(solve2, task_data, verbose=False)
    
    code1 = "solve=lambda g:[[x for j in range(len(g)) for x in (g[s] if g[i][j] else [0]*len(g))] for i in range(len(g)) for s in range(len(g))]"
    code2 = "solve=lambda g:sum([[[0]*len(g)if g[i][j]==0 else g[s]for j in range(len(g))]for i in range(len(g))for s in range(len(g))],[])"
    
    bytes1 = len(code1.encode('utf-8'))
    bytes2 = len(code2.encode('utf-8'))
    
    print(f"\nVersion 1:")
    print(f"  Correctness: {'PASS' if is_opt1_correct else 'FAIL'}")
    print(f"  Byte count: {bytes1}")
    print(f"  Score: {max(1, 2500 - bytes1)} points")
    
    print(f"\nVersion 2:")
    print(f"  Correctness: {'PASS' if is_opt2_correct else 'FAIL'}")
    print(f"  Byte count: {bytes2}")
    print(f"  Score: {max(1, 2500 - bytes2)} points")
    
    # Show the best working version
    if is_opt1_correct:
        print(f"\nBEST SOLUTION (Version 1):")
        print(f"  Code: {code1}")
    elif is_opt2_correct:
        print(f"\nBEST SOLUTION (Version 2):")
        print(f"  Code: {code2}")
    
else:
    print(f"\nFAILURE: {message}")
